In [1]:
import pandas as pd
import numpy as np

# 엑셀 파일 경로 및 시트 이름
file_path = 'Delta_PSM.csv'

# csv 파일 불러오기
df = pd.read_csv(file_path)

# 계산할 컬럼 목록
columns_to_calculate = [
    'X_reg', 'Y_reg', 'pred_x', 'pred_y', 'residual_x', 'residual_y',  
    'psm_fit_x', 'psm_fit_y', 'residual_x_depsm', 'residual_y_depsm',
    'cpe19p_pred_x', 'cpe19p_pred_y', 'cpe19p_resi_x', 'cpe19p_resi_y',
    'ideal_psm_x', 'ideal_psm_y', 'delta_psm_x', 'delta_psm_y' ]

# 반지름 계산
df['radius'] = np.sqrt(df['wf_x']**2 + df['wf_y']**2)

# 영역 할당
conditions = [
    (df['radius'] <= 50000),
    (df['radius'] > 50000) & (df['radius'] <= 100000),
    (df['radius'] > 100000) & (df['radius'] <= 150000)
]
choices = ['Center', 'Middle', 'Edge']
df['region'] = np.select(conditions, choices, default='Outside')

# 'UNIQUE_ID'와 'region' 기준으로 그룹화하여 mean과 std(표준편차) 계산 (mean 후 절대값 적용)
grouped = df.groupby(['UNIQUE_ID', 'region']).agg({col: ['mean', 'std'] for col in columns_to_calculate})

# 컬럼명 재정의 (mean과 std의 멀티인덱스를 단일 인덱스로 변환)
grouped.columns = [f'{col}_{stat}' for col, stat in grouped.columns]

# mean에 절대값 적용
for col in columns_to_calculate:
    grouped[f'{col}_mean'] = grouped[f'{col}_mean'].abs()

# mean + 3 * sigma 계산
for col in columns_to_calculate:
    grouped[f'{col}_m3s'] = grouped[f'{col}_mean'] + 3 * grouped[f'{col}_std']

# 첫 번째 값을 그룹화해서 각각 추가 (UNIQUE_ID와 region 기준)
grouped['STEPSEQ'] = df.groupby(['UNIQUE_ID', 'region'])['STEPSEQ'].first()
grouped['LOT_ID'] = df.groupby(['UNIQUE_ID', 'region'])['LOT_ID'].first()
grouped['Wafer'] = df.groupby(['UNIQUE_ID', 'region'])['Wafer'].first()
grouped['P_EQPID'] = df.groupby(['UNIQUE_ID', 'region'])['P_EQPID'].first()
grouped['Photo_PPID'] = df.groupby(['UNIQUE_ID', 'region'])['Photo_PPID'].first()
grouped['P_TIME'] = df.groupby(['UNIQUE_ID', 'region'])['P_TIME'].first()
# grouped['M_STEP'] = df.groupby(['UNIQUE_ID', 'region'])['M_STEP'].first()
grouped['M_TIME'] = df.groupby(['UNIQUE_ID', 'region'])['M_TIME'].first()
grouped['ChuckID'] = df.groupby(['UNIQUE_ID', 'region'])['ChuckID'].first()
grouped['ReticleID'] = df.groupby(['UNIQUE_ID', 'region'])['ReticleID'].first()
grouped['Base_EQP1'] = df.groupby(['UNIQUE_ID', 'region'])['Base_EQP1'].first()
grouped['GROUP'] = df.groupby(['UNIQUE_ID', 'region'])['GROUP'].first()
grouped['MMO_MRC_EQP'] = df.groupby(['UNIQUE_ID', 'region'])['MMO_MRC_EQP'].first()

# 인덱스 재설정 (UNIQUE_ID와 region을 컬럼으로 변환)
grouped.reset_index(inplace=True)

# 원하는 컬럼 순서 정의
desired_column_order = [
    'UNIQUE_ID', 'region', 'STEPSEQ', 'LOT_ID', 'Wafer', 'P_EQPID', 'Photo_PPID', 'P_TIME',
    'M_TIME', 'ChuckID', 'ReticleID', 'Base_EQP1', 'MMO_MRC_EQP', 'GROUP'
]
# M_STEP은 삭제함

# 계산된 컬럼 추가
for col in columns_to_calculate:
    desired_column_order.extend([
        f'{col}_mean',
        f'{col}_std',
        f'{col}_m3s'
    ])

# DataFrame의 컬럼을 재배열
grouped = grouped[desired_column_order]

# 새로운 파일로 저장 (CSV 형식으로 저장)
output_file_path = 'output_m3s_by_region.csv'
grouped.to_csv(output_file_path, index=False)

print("영역별로 m3s 계산 완료 및 저장됨:", output_file_path)


영역별로 m3s 계산 완료 및 저장됨: output_m3s_by_region.csv
